# Predictive model

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import itertools
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
layers  = keras.layers

In [3]:
print("you have tensorflow version",tf.__version__)

you have tensorflow version 2.7.0


In [4]:
df = pd.read_csv("finalma1.csv")

# Training data set and testing data set


In [5]:
train_size = int(len(df)*.87)
print("Train size: ",train_size)
print("Test size: ",(len(df)-train_size))


Train size:  129256
Test size:  19315


In [6]:
#train features 
cir_train = df['cicuit'][:train_size]
driv_train = df['driverid'][:train_size]

#Train labels
labels_train = df['laptime'][:train_size]

#Test features
cir_test = df['cicuit'][train_size:]
driv_test = df['driverid'][train_size:]

#Test labels
labels_test = df['laptime'][train_size:]

In [7]:
encoder = LabelEncoder()
encoder.fit(cir_train)
cir_train = encoder.transform(cir_train)
cir_test = encoder.transform(cir_test)
num_classes = np.max(cir_train) + 1

cir_train = keras.utils.to_categorical(cir_train,num_classes)
cir_test = keras.utils.to_categorical(cir_test,num_classes)

In [8]:
driversize = df['driverid'].sum()
driversize

34255328

# Functional APIs

In [9]:
bow_inputs = layers.Input(shape=(driversize,))
cir_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs,cir_inputs])
merged_layer = layers.Dense(4,activation='relu')(merged_layer)
predic = layers.Dense(1)(merged_layer)
widemod = keras.Model(inputs=[bow_inputs,cir_inputs],outputs=predic)

In [10]:
widemod.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
widemod.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 34255328)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 34255334)     0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 4)            137021340   ['concatenate[0][0]']        

In [11]:
deep_input = layers.Input(shape=(2000,))
embedding = layers.Embedding(driversize,3,input_length=2000)(deep_input)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs = deep_input,outputs=embed_out)
deep_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2000)]            0         
                                                                 
 embedding (Embedding)       (None, 2000, 3)           102765984 
                                                                 
 flatten (Flatten)           (None, 6000)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6001      
                                                                 
Total params: 102,771,985
Trainable params: 102,771,985
Non-trainable params: 0
_________________________________________________________________


In [12]:
deep_model.compile(loss='mse',optimizer='adam',metrics='accuracy')

In [13]:
merged_out = layers.concatenate([widemod.output,deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(widemod.input+[deep_model.input],merged_out)

combined_model.compile(loss='mse',optimizer='adam',metrics='accuracy')

In [ ]:
tokenize = keras.preprocessing.text.Tokenizer

In [1]:
combined_model.fit([driv_train,cir_train],[],labels_train,epochs=5,batch_size=128)

NameError: name 'combined_model' is not defined

# Run training

Epoch 1/5


ValueError: in user code:

    File "C:\Users\Himadhith\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Himadhith\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Himadhith\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Himadhith\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Himadhith\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C: